In [1]:
import findspark
findspark.init('C:\Spark\spark-3.0.1-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
data = spark.read.csv('customer_churn.csv',
                      inferSchema=True,
                      header=True)

In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(
    inputCols=['Age',
               'Total_Purchase',
               'Account_Manager',
               'Years',
               'Num_Sites'],
    outputCol='features'
)

In [9]:
output = assembler.transform(data)

In [10]:
final_data = output.select('features', 'Churn')

In [11]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [12]:
from pyspark.ml.classification import LogisticRegression

In [13]:
lr_churn = LogisticRegression(labelCol='Churn')

In [14]:
fitted_churn_model = lr_churn.fit(train_churn)

In [15]:
training_sum = fitted_churn_model.summary

In [16]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                645|                645|
|   mean|0.16279069767441862|0.11937984496124031|
| stddev| 0.3694611166056839| 0.3244865795011769|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [18]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [19]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[1.09404939715882...|[0.74914348227870...|       0.0|
|[28.0,11128.95,1....|    0|[4.53676546719257...|[0.98940546025146...|       0.0|
|[29.0,8688.17,1.0...|    1|[3.00218240759165...|[0.95267262338332...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.59151366356385...|[0.98996423559488...|       0.0|
|[29.0,10203.18,1....|    0|[4.10000446119085...|[0.98369757217130...|       0.0|
|[29.0,12711.15,0....|    0|[5.63232976015812...|[0.99643254774116...|       0.0|
|[29.0,13255.05,1....|    0|[4.57729208782548...|[0.98982195453418...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.24635164825776...|[0.96254179275560...|       0.0|
|[30.0,10960.52,1....|    0|[2.79973122428313...|[0.94266129823989...|       0.0|
|[30.0,12788.37,

In [20]:
churn_eval = BinaryClassificationEvaluator(
    labelCol='Churn',
    rawPredictionCol='prediction'
)

In [21]:
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

0.776984126984127

<h3> Predict on New Data </h3>

In [22]:
final_lr_model = lr_churn.fit(final_data)

In [23]:
new_customers = spark.read.csv('new_customers.csv',
                               inferSchema=True,
                               header=True)

In [24]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [25]:
test_new_customers = assembler.transform(new_customers)

In [26]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [27]:
final_results = final_lr_model.transform(test_new_customers)

In [28]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [29]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                  6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|               null|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|         